In [13]:
import pygame
import pygame.midi
from time import sleep

def midiExample():
    GRAND_PIANO = 0
    CHURCH_ORGAN = 19
    instrument = CHURCH_ORGAN
    #instrument = GRAND_PIANO
    
    pygame.init()
    pygame.midi.init()

    port = pygame.midi.get_default_output_id()
    print ("using output_id :%s:" % port)
    midi_out = pygame.midi.Output(port, 0)
    try:
        midi_out.set_instrument(instrument)

        midi_out.note_on(72,127) # 74 is middle C, 127 is "how loud" - max is 127
        sleep(.5)
        midi_out.note_off(72,127)
        sleep(.5)

        midi_out.note_on(76,127) # E
        sleep(.5)
        midi_out.note_off(76,127)
        sleep(.5)

        midi_out.note_on(79,127) # G
        sleep(.5)
        midi_out.note_off(79,127)
        sleep(.5)

        midi_out.note_on(72,127)
        midi_out.note_on(76,127)
        midi_out.note_on(79,127)
        sleep(.5)
        midi_out.note_off(72,127)
        midi_out.note_off(76,127)
        midi_out.note_off(79,127)

    finally:
        del midi_out
        pygame.midi.quit()

midiExample()

using output_id :0:


In [1]:
from mido import MidiFile

mid = MidiFile(r'D:\code\py\MMMake\generated_midi\cnew_song.mi')

for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(msg)
        
        


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\code\\py\\MMMake\\generated_midi\\cnew_song.mi'

In [15]:
from mido import MidiFile

mid = MidiFile(r'D:\code\py\MMMake\midi\testmidi.mid')

for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(msg)

Track 0: 
<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message end_of_track time=0>
Track 1: 
<meta message set_tempo tempo=461538 time=0>
<meta message set_tempo tempo=461538 time=0>
<meta message set_tempo tempo=461538 time=0>
<meta message end_of_track time=0>
Track 2: MIDI Out
<meta message track_name name='MIDI Out' time=0>
control_change channel=0 control=10 value=64 time=0
control_change channel=0 control=7 value=100 time=0
pitchwheel channel=0 pitch=0 time=0
control_change channel=0 control=101 value=0 time=0
control_change channel=0 control=100 value=0 time=0
control_change channel=0 control=6 value=12 time=0
control_change channel=0 control=10 value=64 time=0
control_change channel=0 control=7 value=100 time=0
pitchwheel channel=0 pitch=0 time=0
program_change channel=0 program=0 time=0
control_change channel=0 control=10 value=64 time=0
control_change channel=0 control=7 value=100 time=0
pitchwheel cha

In [20]:
48/96

0.5

In [2]:
from mido import MidiFile, MidiTrack, Message, MetaMessage
import mido

def cal_tick_to_beat(time, ticks):
    if time/ticks <= 0.0625:
        beat = 64
    elif time/ticks <= 0.125:
        beat = 32
    elif time/ticks <= 0.25:
        beat = 16
    elif time/ticks <= 0.5:
        beat = 8
    elif time/ticks <= 1:
        beat = 4
    elif time/ticks <= 2:
        beat = 2
    elif time/ticks <= 4:
        beat = 1
    else:
        beat = 0.5
    return beat


def cal_beat_to_tick(beat, ticks):
    cal = int((ticks*4)/int(beat))
    return cal

bpm = 140
beat = 480

track_list = []

ticks = 0

element = ''
list = []
notelist = []
timelist = []
onlist = []
ListOfList = []
off_flag = False
ontime = 0

mid_analysis = MidiFile(r'D:\code\py\MMMake\midi\alstroemeria.mi')

ticks = mid_analysis.ticks_per_beat

for i, track in enumerate(mid_analysis.tracks):
    print('Track {}: {}'.format(i, track.name))    
    for msg in track:
        if msg.type == 'note_on':
            if msg.time != 0:
                list.append('r'+str(cal_tick_to_beat(msg.time, ticks)))
                for c, out in enumerate(onlist):
                    element += str(out) + '-' + str(cal_tick_to_beat(msg.time, ticks))
                    #element += str(out) + '-' + str(msg.time)
                    if c == len(onlist)-1: 
                        list.append(element)
                        element = ''
                        onlist = []
                    else:
                        element += ','                  
            onlist.append(msg.note)
        if msg.type == 'note_off':
            if onlist:
                for c, out in enumerate(onlist):
                    element += str(out) + '-' + str(cal_tick_to_beat(msg.time, ticks))
                    #element += str(out) + '-' + str(msg.time)
                    if c == len(onlist)-1: 
                        list.append(element)
                        element = ''
                        onlist = []
                    else:
                        element += ','
                    
    ListOfList.append(list)
    list = []

                  
for i in ListOfList:
    for j in i:
        print(j, end=' ')
        print()
    print()


#파싱 미디파일 출력
mid_create = MidiFile(ticks_per_beat = ticks)
for i in range(len(ListOfList)):
    track_list.append(MidiTrack())

mid_create.tracks.append(track_list[0])

track_list[0].append(Message('program_change', program=0, time=0))
track_list[0].append(MetaMessage('set_tempo', tempo=mido.bpm2tempo(bpm), time=0))

for count, i in enumerate(ListOfList):
    if count != 0:
        mid_create.tracks.append(track_list[count])
    for j in i:
        if j[0] == 'r':
            ontime += int(j[1:])
        else:
            temp = []
            notelist = []
            timelist = []
        
            temp = j.split(',')
        
            for k in temp:
                for h in range(len(k)):
                    if k[h] == '-':
                        notelist.append(k[:h])
                        timelist.append(k[h+1:])

            for k in notelist:
                track_list[count].append(Message('note_on', note=int(k), velocity=100, time=ontime))

            for h,k in enumerate(timelist):
                if h != 0:
                    if timelist[h-1] == timelist[h]:
                        track_list[count].append(Message('note_off', note=int(notelist[h]), velocity=100, time=0))
                else:        
                    track_list[count].append(Message('note_off', note=int(notelist[h]), velocity=100, time=cal_beat_to_tick(k, ticks)))
            ontime = 0
    track_list[count].append(MetaMessage('end_of_track'))
    
mid_create.save('D:\code\py\MMMake\generated_midi\cnew_song.mid')
print('Done')



FileNotFoundError: [Errno 2] No such file or directory: 'D:\\code\\py\\MMMake\\midi\\alstroemeria.mi'

In [25]:
from mido import MidiFile, MidiTrack, Message, MetaMessage
import mido

major_list = [0, 2, 4, 5, 7, 9, 11, 12]
minor_list = [0, 2, 3, 5, 7, 9, 10, 12]

c4 = 60

list = []
for i in major_list:
    list.append(i+c4)

    
bpm = 120
beat = 480

mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

track.append(Message('program_change', program=1, time=0))
track.append(MetaMessage('set_tempo', tempo=mido.bpm2tempo(bpm), time=0))


for i in list:
    track.append(Message('note_on', note=i, velocity=100, time=0))
    track.append(Message('note_off', note=i, velocity=100, time=beat))
    
track.append(Message('note_off', note=i, velocity=100, time=beat))
    
for i in reversed(list):
    track.append(Message('note_on', note=i, velocity=100, time=0))
    track.append(Message('note_off', note=i, velocity=100, time=beat))
    

mid.save('new_song.mid')

port = mido.open_output()

mid = mido.MidiFile('new_song.mid')
for msg in mid.play():
    port.send(msg)
    
for msg in mid:
    print(msg)
    
port.close()
if port.closed:
    print("Yup, it's closed.")

program_change channel=0 program=1 time=0
<meta message set_tempo tempo=500000 time=0>
note_on channel=0 note=60 velocity=100 time=0
note_off channel=0 note=60 velocity=100 time=0.5
note_on channel=0 note=62 velocity=100 time=0
note_off channel=0 note=62 velocity=100 time=0.5
note_on channel=0 note=64 velocity=100 time=0
note_off channel=0 note=64 velocity=100 time=0.5
note_on channel=0 note=65 velocity=100 time=0
note_off channel=0 note=65 velocity=100 time=0.5
note_on channel=0 note=67 velocity=100 time=0
note_off channel=0 note=67 velocity=100 time=0.5
note_on channel=0 note=69 velocity=100 time=0
note_off channel=0 note=69 velocity=100 time=0.5
note_on channel=0 note=71 velocity=100 time=0
note_off channel=0 note=71 velocity=100 time=0.5
note_on channel=0 note=72 velocity=100 time=0
note_off channel=0 note=72 velocity=100 time=0.5
note_off channel=0 note=72 velocity=100 time=0.5
note_on channel=0 note=72 velocity=100 time=0
note_off channel=0 note=72 velocity=100 time=0.5
note_on c

In [105]:
list = [0]
llist = [0]
llist.append(list)

llist

[0, [0]]

In [3]:
from mido import MidiFile, MidiTrack, Message, MetaMessage
import mido
import time

mid_create = MidiFile()
track = MidiTrack()
mid_create.tracks.append(track)

for msg in MidiFile(r'D:\code\py\mid\alstroemeria.mi'):
    time.sleep(msg.time)
    if not msg.is_meta:
        print(msg)

mid_create.save('Test.mid')
print('ddd')


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\code\\py\\mid\\alstroemeria.mi'

In [4]:
from mido import MidiFile

testFile = MidiFile()
testFile.ticks_per_beat

480

In [ ]:
from mido import MidiFile, MidiTrack, Message, MetaMessage
import mido

bpm = 120
beat = 480

track_list = []

ticks = 0

element = ''
list = []
notelist = []
timelist = []
onlist = []
ListOfList = []
off_flag = False

mid_analysis = MidiFile(r'D:\code\py\MMMake\midi\testmidi.mid')

ticks = mid_analysis.ticks_per_beat

for i, track in enumerate(mid_analysis.tracks):
    print('Track {}: {}'.format(i, track.name))    
    for msg in track:
        if msg.type == 'note_on':
            onlist.append(msg.note)
        if msg.type == 'note_off':
            onlist.remove(msg.note)
            element += str(msg.note) + '-' + str(msg.time)
            if not onlist: 
                list.append(element)
                element = ''
            else:
                element += ','
    ListOfList.append(list)
    list = []

                  
for i in ListOfList:
    for j in i:
        print(j, end=' ')
        print()
    print()

'''
#파싱 미디파일 출력
mid_create = MidiFile(ticks_per_beat = ticks)
for i in range(len(ListOfList)):
    track_list.append(MidiTrack())

mid_create.tracks.append(track_list[0])

track_list[0].append(Message('program_change', program=0, time=0))
track_list[0].append(MetaMessage('set_tempo', tempo=mido.bpm2tempo(bpm), time=0))

for count, i in enumerate(ListOfList):
    if count != 0:
        mid_create.tracks.append(track_list[count])
    for j in i:
        temp = []
        notelist = []
        timelist = []
        temp = j.split(',')
        
        for k in temp:
            for h in range(len(k)):
                if k[h] == '-':
                    notelist.append(k[:h])
                    timelist.append(k[h+1:])
                    
        for k in notelist:
            track_list[count].append(Message('note_on', note=int(k), velocity=100, time=0))

        for h,k in enumerate(timelist):
            track_list[count].append(Message('note_off', note=int(notelist[h]), velocity=100, time=int(k)))
    track_list[count].append(MetaMessage('end_of_track'))
    
mid_create.save('D:\code\py\MMMake\generated_midi\cnew_song.mid')
print('Done')
'''


In [17]:
from mido import MidiFile, MidiTrack, Message, MetaMessage
import mido

class note:
    note = -1
    ontime = -1
    endtime = -1
    note_string = ''
    def __init__(self, note, ontime):
        self.note = note
        self.ontime = ontime
    def __del__(self):
        print('삭제')
    def make_note(self, endtime):
        #if endtime != 0:
        self.note_string += str(self.note) + '-' + str(endtime)

class rest_note(note):
    def __init__(self, ontime):
        self.note_string += 'r' + '-' + str(ontime)

    

def cal_tick_to_beat(time, ticks):
    if time/ticks <= 0.0625:
        beat = 64
    elif time/ticks <= 0.125:
        beat = 32
    elif time/ticks <= 0.25:
        beat = 16
    elif time/ticks <= 0.5:
        beat = 8
    elif time/ticks <= 1:
        beat = 4
    elif time/ticks <= 2:
        beat = 2
    elif time/ticks <= 4:
        beat = 1
    else:
        beat = 0.5
    return beat


def cal_beat_to_tick(beat, ticks):
    cal = int((ticks*4)/int(beat))
    return cal



bpm = 140
beat = 480

track_list = []

ticks = 0

element = ''
list = []
notelist = []
timelist = []
onlist = []
ListOfList = []
off_flag = False
ontime = 0


mid_analysis = MidiFile(r'D:\code\py\MMMake\midi\testmidi.mid')

ticks = mid_analysis.ticks_per_beat

for i, track in enumerate(mid_analysis.tracks):
    print('Track {}: {}'.format(i, track.name))    
    for msg in track:
        if msg.type == 'note_on':
            ob_note = note(msg.note,msg.time)
            if ob_note.ontime != 0:
                list.append(onlist)
                #쉼표넣기

                
                onlist = []
            onlist.append(ob_note)
        if msg.type == 'note_off':
            list.append(onlist)
            for k in list:
                for comp in k:
                    if comp.note_string == '':
                        if comp.note == msg.note:
                            comp = comp.make_note(msg.time)
                            break    
            onlist = []
    ListOfList.append(list)
    list = []

                
for i in ListOfList:
    for j in i:
        for k in j:
            print(k.note_string, end=' ')
            print()
        print()
    print()


'''
#파싱 미디파일 출력
mid_create = MidiFile(ticks_per_beat = ticks)
for i in range(len(ListOfList)):
    track_list.append(MidiTrack())

mid_create.tracks.append(track_list[0])

track_list[0].append(Message('program_change', program=0, time=0))
track_list[0].append(MetaMessage('set_tempo', tempo=mido.bpm2tempo(bpm), time=0))

for count, i in enumerate(ListOfList):
    if count != 0:
        mid_create.tracks.append(track_list[count])
    for j in i:
        if j[0] == 'r':
            ontime += int(j[1:])
        else:
            temp = []
            notelist = []
            timelist = []
        
            temp = j.split(',')
        
            for k in temp:
                for h in range(len(k)):
                    if k[h] == '-':
                        notelist.append(k[:h])
                        timelist.append(k[h+1:])

            for k in notelist:
                track_list[count].append(Message('note_on', note=int(k), velocity=100, time=ontime))

            for h,k in enumerate(timelist):
                if h != 0:
                    if timelist[h-1] == timelist[h]:
                        track_list[count].append(Message('note_off', note=int(notelist[h]), velocity=100, time=0))
                else:        
                    track_list[count].append(Message('note_off', note=int(notelist[h]), velocity=100, time=cal_beat_to_tick(k, ticks)))
            ontime = 0
    track_list[count].append(MetaMessage('end_of_track'))
    
mid_create.save('D:\code\py\MMMake\generated_midi\cnew_song.mid')
print('Done')
'''

삭제
삭제
삭제
삭제
삭제
삭제
삭제
삭제
삭제
삭제
삭제
삭제
Track 0: 
Track 1: 
Track 2: MIDI Out
삭제


TypeError: 'rest_note' object is not iterable